In [91]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd
from scipy.interpolate import interp1d
from enum import Enum
from numpy.linalg import norm

In [80]:
class InputError(Exception):
    pass

class ErrorDeCarga(Exception):
    pass


class TipoElectrodo(Enum):
    punto = 'punto'
    circulo = 'circulo'
    rectangulo = 'rectangulo'

class DistribucionNervio(Enum):
    uniforme = 'uniforme',
    gaussiana = 'gaussiana'
    manual = 'manual'

class Nervio:
    def __init__(self,diametro = 250):
        self.diametro = diametro
        self.fibras = []
        self.cantidad_fibras = 0
    
    def poblar(self,metodo_poblacion='uniforme',num_fibras=200,semilla=None,**kwargs):
        if metodo_poblacion == DistribucionNervio.uniforme.name:
            self.distribucion_uniforme(num_fibras,semilla)
        elif metodo_poblacion == DistribucionNervio.gaussiana.name:
            self.distribucion_normal(num_fibras,semilla)
        elif metodo_poblacion == DistribucionNervio.manual.name:
            self.cargar_archivo(kwargs.get('nombre_archivo',None))
        else:
            raise InputError ('El método de población ingresado no es correcto.')

    def distribucion_uniforme(self,num_fibras,semilla = None):
        np.random.seed(semilla)
        radio = self.diametro/2
        for i in range(num_fibras):
            length = np.random.uniform(0,radio**2)
            angle = np.pi * np.random.uniform(0, 2) #de o a pi
            x = np.sqrt(length) * np.cos(angle)
            y = np.sqrt(length) * np.sin(angle)
            self.fibras.append(Fibra(x,y))
        self.cantidad_fibras = num_fibras
    
    def distribucion_normal(self,num_fibras,semilla = None):
        np.random.seed(semilla)
        radio = self.diametro/2
        for i in range(num_fibras):
            length = np.random.normal(0,radio)
            angle = np.pi * np.random.uniform(0, 2)
            while np.abs(length) > radio:
                length = np.random.normal(0,radio)
            x = length * np.cos(angle)
            y = length * np.sin(angle)
            self.fibras.append(Fibra(x,y))
        self.cantidad_fibras = num_fibras
    
    def cargar_archivo(self,nombre_archivo):
        try:
            fibras = pd.read_csv(nombre_archivo)
        except Exception as e:
            raise ErrorDeCarga('Fallo en carga de archivo.') from e
        cantidad_fibras = len(fibras)
        for i in range(0,cantidad_fibras):
            x = fibras["x"][i]
            y = fibras["y"][i]
            self.fibras.append(Fibra(x,y))
        self.cantidad_fibras = cantidad_fibras
    
    def ploteo_distribucion_fibras(self):
        lista_x = []
        lista_y = []
        for f in self.fibras:
            x = f.x
            y = f.y
            lista_x.append(x)
            lista_y.append(y)
        plt.figure(figsize = (5,5))
        plt.plot(lista_x,lista_y,"r .")
    
    def __repr__(self):
        return f'Nervio(diametro={self.diametro},num_fibras={self.cantidad_fibras})'

class Fibra:
    def __init__(self,x,y):
        self.x = x
        self.y = y

class Electrodo:
    def __init__(self,tipo,**kwargs):
        try:
            self.tipo = TipoElectrodo(tipo)
        except ValueError as e:
            raise InputError('El tipo de electrodo ingresado no es correcto.') from e
        self.dist_al_electrodo = []

        if self.tipo == TipoElectrodo.punto:
            self.x = kwargs.get('x')
            self.y = kwargs.get('y')
            if self.x is None or self.y is None:
                raise Exception('Los parámetros x e y son necesarios para electrodo tipo punto.')
        elif self.tipo == TipoElectrodo.circulo:
            self.x = kwargs.get('x')
            self.y = kwargs.get('y')
            self.radio = kwargs.get('radio')
            if self.x is None or self.y is None or self.radio is None:
                raise Exception('Los parámetros x, y, radio son necesarios para el electrodo tipo circulo.')
        elif self.tipo == TipoElectrodo.rectangulo:
            self.lado = kwargs.get('lado')
            if self.lado is None:
                raise Exception('El parámetro lado es necesario para el electrodo tipo rectángulo.')

    def calcular_distancias(self,lista_fibras,**kwargs):
        if self.tipo == TipoElectrodo.punto:
            self.distancia_al_electrodo_punto(lista_fibras,**kwargs)
        elif self.tipo == TipoElectrodo.circulo:
            self.distancia_al_electrodo_circulo(lista_fibras,**kwargs)
        elif self.tipo == TipoElectrodo.rectangulo:
            self.distancia_al_electrodo_rectangulo(lista_fibras,**kwargs)
        
    
    def distancia_al_electrodo_punto(self,lista_fibras: list[Fibra]):
        for f in lista_fibras:
            self.dist_al_electrodo.append(np.linalg.norm([f.x-self.x,f.y-self.y]))
    
    def distancia_al_electrodo_circulo(self,lista_fibras: list[Fibra]):
        for fibra in lista_fibras:
            self.dist_al_electrodo.append(np.linalg.norm([fibra.x-self.x,fibra.y-self.y])-self.radio)
    
    def distancia_al_electrodo_rectangulo(self,lista_fibras: list[Fibra],diametro):
        a = np.array([-self.lado/2,-diametro/2])
        b = np.array([self.lado/2,-diametro/2])
        for fibra in lista_fibras:
            p = np.array([fibra.x,fibra.y])
            if np.arccos(np.dot((p-a)/norm(p-a), (b-a)/norm(b-a))) > np.pi/2:
                d = norm(p-a)
            elif np.arccos(np.dot((p-b)/norm(p-b), (a-b)/norm(a-b))) > np.pi/2:
                d = norm(p-b)
            else:
                d = norm(np.cross(b-a, a-p))/norm(b-a)
            self.dist_al_electrodo.append(d)


    def __repr__(self):
        return f'Electrodo ({self.tipo})'




class Entorno:
    def __init__(self):
        self.nervio = None
        self.electrodos = []
    
    def crear_nervio(self,num_fibras=200,diametro=250,metodo_poblacion='uniforme',semilla=None,**kwargs):
        self.nervio = Nervio(diametro)
        self.nervio.poblar(num_fibras=num_fibras,metodo_poblacion=metodo_poblacion,semilla=semilla,**kwargs)
    
    def dibujar_nervio(self):
        self.nervio.ploteo_distribucion_fibras()
    
    def crear_electrodo(self,tipo,**kwargs):
        self.electrodos.append(Electrodo(tipo,**kwargs))

    def distancia_al_electrodo(self):
        for electrodo in self.electrodos:
            if electrodo.tipo == TipoElectrodo.rectangulo:
                electrodo.calcular_distancias(lista_fibras = self.nervio.fibras,diametro = self.nervio.diametro)
            else:
                electrodo.calcular_distancias(lista_fibras = self.nervio.fibras)
    
    def imprimir_distancias(self):
        for electrodo in self.electrodos:
            print(electrodo.dist_al_electrodo)



In [90]:
a = Entorno()
a.crear_nervio(metodo_poblacion='uniforme')
#a.dibujar_nervio()
a.crear_electrodo(tipo = 'punto',x=0,y=-125)
a.crear_electrodo(tipo='punto',x = 0, y = 125)
a.distancia_al_electrodo()
#a.imprimir_distancias()
